## Ordnance Survey Data Hub Tutorials - Example Applications

# Using the OS Features API, OS Linked Identifier API and OS Places API with ipyleaflet (OS-PAW version)

This is a Python/Jupyter Notebook application, demonstrating how the OS Features API, OS Linked Identifiers API and OS Places API can be made to work together within an interactive ipyleaflet (https://ipyleaflet.readthedocs.io/en/latest/) map.  The application also makes use of the OS Maps API, as a source of contextual background mapping. 

This example initially queries and displays Topo Area building features surrounding a specified point (in this example, the town of Glossop), using the OS Features API.  When the user then elects to click upon one of the building polygons, the application queries the OS Linked Identifiers API to find a UPRN code that relates to that building (via it's toid identifier).  If this is sucessful, that feature's address is then able to be queried from the OS Places API, which is then displayed in an ipyleaflet Popup layer.

This demo also uses OS-PAW (available from PyPI), a tool that greatly simplifies working with the OS Features API, when in a Python environment.

The four APIs used in this application are all available from the OS Data Hub https://osdatahub.os.uk/

In [ ]:
from datetime import datetime
from ipyleaflet import Map, WidgetControl, GeoJSON, Popup
from ipywidgets import Image, HTML
from urllib.parse import unquote, urlencode
from pyproj import Transformer, crs
from os_paw.wfs_api import WFS_API
import requests
import IPython
import geopandas as gpd
import random
import winsound  # Replace with equivalent, if using OSX or Linux
import platform


Use of the OS APIs requires a service key. Sign up for yours at https://osdatahub.os.uk/ and enter it below.

In [ ]:
# OS Data Hub project API key
key = 'YOUR API KEY HERE'

Define the location and extent of the query here.

In [ ]:
# Centre of map - define default starting location
centreOfMapBNG = {"eastings" : 403496 , "northings" : 394064}  # This default location is for Glossop High Street.
#centreOfMapBNG = {"eastings" : 654810 , "northings" : 293154}  # An alternative test location, for Lowestoft.

# AOI dimensions (meters)
aoi_dimensions = {"width" : 400, "height" : 300}

Calculate the area of interest, converting both that, and the centre location, to WGS84 for onward use.

In [ ]:
# General BNG to WGS84 convertor, using pyproj
def convertBNGtoLatLon(eastings, northings):

    # Convert a BNG coordinate pair to lat/lon
    bng = crs.CRS('epsg:27700')
    wgs84 = crs.CRS('epsg:4326')

    transformer = Transformer.from_crs(bng, wgs84)
    wgs84_point = transformer.transform(eastings, northings)

    return wgs84_point

# Calculate area of interest
aoi_SW_BNG = {"eastings" : centreOfMapBNG.get("eastings") - (aoi_dimensions.get("width")/2), "northings" : centreOfMapBNG.get("northings") - (aoi_dimensions.get("height")/2)}
aoi_NE_BNG = {"eastings" : centreOfMapBNG.get("eastings") + (aoi_dimensions.get("width")/2), "northings" : centreOfMapBNG.get("northings") + (aoi_dimensions.get("height")/2)}

# Convert to WGS84
centreOfMap = convertBNGtoLatLon( centreOfMapBNG.get("eastings"), centreOfMapBNG.get("northings"))

aoi_SW_WGS84 = convertBNGtoLatLon( aoi_SW_BNG.get("eastings"), aoi_SW_BNG.get("northings"))
aoi_NE_WGS84 = convertBNGtoLatLon( aoi_NE_BNG.get("eastings"), aoi_NE_BNG.get("northings"))

As with most ipyleaflet projects that we're likely to be building, we'll start with creating a map object and a background layer.  For the latter, we'll use the OS Maps API.

In [ ]:
# Function to create ipyleaflet tile layers
def createOSMapsTileLayer(layertype):

    params = urlencode({'key': key,
                        'service': 'WMTS',
                        'request': 'GetTile',
                        'version': '2.0.0',
                        'height': 256,
                        'width': 256,
                        'outputFormat': 'image/png',
                        'style': 'default',
                        'layer': layertype,
                        'tileMatrixSet': 'EPSG:3857',
                        'tileMatrix': '{z}',
                        'tileRow': '{y}',
                        'tileCol': '{x}'})

    # OS Data Hub base path - https://api.os.uk
    # OS Maps API WMTS end point path - /maps/raster/v1/wmts?
    url = unquote(f'https://api.os.uk/maps/raster/v1/wmts?{params}')

    tileLayer = {'url': url,
               'min_zoom': 7,
               'max_zoom': 20,
               'attribution': f'Contains OS data &copy; Crown copyright and database rights {datetime.now().year}'}

    return tileLayer

In [ ]:
# Create custom ipyleaflet TileLayer for the OS Maps API WMTS resource
os_maps_api_light = createOSMapsTileLayer('Light_3857')

In [ ]:
# Create ipyleaflet Map
m = Map(basemap=os_maps_api_light,
        center=centreOfMap,
        zoom=16)

...also, add our OS logo, as a widget.

In [ ]:
# Add an image overlay, placing the OS logo in the bottom left corner.
oslogo_url = 'https://raw.githubusercontent.com/OrdnanceSurvey/os-api-branding/master/img/os-logo-maps.png'
oslogo_img = IPython.display.Image(oslogo_url, width = 300)
oslogoWidget = Image(value=oslogo_img.data, format='png', width=100, height=30)
widgetControl = WidgetControl(widget=oslogoWidget, position='bottomleft')
m.add_control(widgetControl)

As this project will include some click-based interactivity that also triggers further API calls.  By adding a quick beep noise, we can reassure users that the click was actually triggered, should there be any delay in the API response.  Python beep calls are platform dependent.  The code, below, is for Windows, so an equivalent will be needed if you're working on other platforms.

In [ ]:
# Play a beep noise (platform dependent)
def make_beep(pitch, length):

    thisPlatform = platform.system()

    if thisPlatform == "Windows":       # Replace with OSX/Linux equivalents, if required.
        winsound.Beep(pitch, length)

    return


The process of requesting JSON from the OS APIs takes a similar form in each case.  So, in the example, we might as well make a function for that.

In [ ]:
# A common API requestor 
def json_api_requestor(url, serviceName):

    try:
        response = requests.get(url)
    except:
        raise Exception("Unable to query and process " + serviceName + ".")

    try:
        jsonResult = response.json()
    except:
        raise Exception("Unable to convert the returned payload to JSON.")

    return jsonResult

To relate the area feature with an address, we'll use a UPRN as the common link.  We can obtain this from the OS Linked Identifiers API.

In [ ]:
# For a given toid, find the associated UPRN, using the OS Linked Identifiers API
def get_uprn(toid):

     # Use the OS Linked Identifiers APi to obtain the UPRN for this feature (should be a building!)
    linkapi_url = 'https://api.os.uk/search/links/v1/identifierTypes/TOID/' + toid + '?key=' + key

    # Action the API request
    linkdata = json_api_requestor(linkapi_url, "OS Linked Identifiers API")

    # Now extract the UPRN from the returned JSON file ().  The Linked Identifier API returns a dictionary containing two primary
    # parts - the 'linkededIdentifier', which confirms details of the provided input parameters, and the 'correlations', which is
    # a list of records that, of course, correlate with those parameters.  For this exercise, we're only looking for associated 
    # UPRN records...and we only really need one for each toid that is supplied (which is likely all that will be returned).

    uprn = "Null"
    
    for x in linkdata.get("correlations"):

         # Check that this correlation record pertains to a UPRN
         if x.get("correlatedIdentifierType") == "UPRN":
             uprn = x.get("correlatedIdentifiers")[0].get("identifier")  # Another list, so we'll just take the first entry.
             break

    return uprn
    

Once we've obtained a valid UPRN, we can use it to query the OS Places API, in this case for a postal address.  For this example, we'll just use the Royal Mail's Delivery Point as the source of the addresses, although using the local authority (LPI) alternatives would be a valid option here, too.

In [ ]:
# For a given UPRN, get a postal address from the OS Places API
def get_address_details(uprn):

    # Define the OS Places API URL
    base_osplaces_url = 'https://api.os.uk/search/places/v1/'
    osplaces_path = f'uprn?uprn={uprn}'
    placesURL = f'{base_osplaces_url}{osplaces_path}&format=json&dataset=DPA&output_srs=WGS84&key={key}'  # We'll just use the DPA data for this demo.

    # Action the API request
    placesData = json_api_requestor(placesURL, "OS Places API")

    # Extract the address details we're intersted in
    try:
        postalAddress = placesData.get("results")[0].get("DPA").get("ADDRESS")
    except:
        postalAddress = 'No Delivery Point address found.' # In case this feature has no DPA address


    return postalAddress

We want users to be able to click upon the map features, to see their postal address.  These features will all be present within an ipyleaflet GeoJSON layer.  That layer, by default, has an on_click() method, which we'll configure to use with the following function.  The three arguments, received by the function, are outlined in the comment below.  To understand their content, it's worth writing their content into a file, for further reference.

In [ ]:
# Show details upon feature click
def click_handler(**kwargs):

    # Three arguments should be returned for each WFS feature clicked.  These are
    #
    #   'event'       -  returns just a "click" text string
    #   'feature'     -  returns a complex dictionary object, including both the geometry and the attribution
    #   'properties'  -  returns just the attribution in a simpler dictionary

    # Uncomment this block to write the on_click() args to a file, if you need to examine them.
    # f = open("Test_seeTheArgs.txt", "a")
    # for key, value in kwargs.items():
    #     f.write("%s == %s" %(key, value))
    # f.close()

    # Beep!  Give extra feedback on feature-click.
    make_beep(440, 200) 

    attribution = kwargs.get("properties")
    toid = attribution.get("TOID")

    # Get a point location from the geometry.  We'll use this to position the Popup feature that will hold the address details.
    thisFeature = kwargs.get('feature')
    thisGeom = thisFeature.get('geometry')
    thisCoordsList = thisGeom.get('coordinates')
    firstVertexR = thisCoordsList[0][0]
    firstVertex = (firstVertexR[1],firstVertexR[0])

    # Obtain the UPRN for this toid.
    thisUPRN = get_uprn(toid)

    # Get address details for this UPRN
    if thisUPRN != "Null":
        thisAddress = get_address_details(thisUPRN)
    else:
        thisAddress = "No UPRN found."

    # Define the pop-up message
    message = HTML()
    message.value = thisAddress

    # Add a pop-up
    popup = Popup(
        location=firstVertex,
        child=message,
        close_button=False,
        auto_close=False,
        close_on_escape_key=False
    )

    m.add_layer(popup)


A little callback for assigning each of the building polygons a random colour.

In [ ]:
# Used for the GeoJSON style callback
def random_color(feature):
    return {
        'color': 'black',
        'fillColor': random.choice(['red', 'yellow', 'green', 'orange', 'pink', 'blue']),
    }

This is where we query the OS Features API for building polygons and add them into a map layer, which is then enabled for interactive clicking.  In this version, we'll use the original OS-PAW package, which makes it much easier to work with the OS Features API within Python.

In [ ]:
# Create WFS API instance 
wfs_api = WFS_API(api_key=key)

# Undertake the query itself.
TopoAreaData = wfs_api.get_all_features_within_bbox(type_name="Topography_TopographicArea",
                                            bbox="53.442538,-1.9528413, 53.444864,-1.9462860",
                                            srs='EPSG:4326',
                                            allow_premium=True,
                                            max_feature_count=2000
                                           )

# OS-PAW doesn't support filtering, but we can easily do that in a geopandas GeoDataFrame
gdf = gpd.GeoDataFrame.from_features(TopoAreaData, crs=TopoAreaData['crs'])

buildings_gdf = gdf[gdf['Theme'].str.contains('Buildings', na=False)]


In [ ]:
# Convert the filtered GeoDataFrame back to a GeoJSON dictionary
results_geojson = buildings_gdf.to_json(na='drop')
results_dict = eval(results_geojson)

# Create an ipyleaflet GeoJSON object from the building features...
geo_json = GeoJSON(
    data=results_dict,  # this needs to be dictionary
    style={'opacity': 1, 'dashArray': '0', 'fillOpacity': 0.1, 'weight': 1},
    hover_style={'color': 'white', 'dashArray': '0', 'fillOpacity': 0.5},
    style_callback=random_color
)

# ...and add the layer to our map.
m.add_layer(geo_json)

# Also, enable the GeoJSON layer's on_click() method, so we can show each feature's address, should it have one.
geo_json.on_click(click_handler)


In [ ]:
# Display the map
m